In [1]:
import glob
import geopandas as gpd
import sys
import os
import pandas as pd
import numpy as np
import rasterio as rio
from rasterio.features import rasterize
import rasterio.windows
from matplotlib import pyplot as plt
from tqdm import tqdm
sys.path.append('/home/sushen/marine_debris_semester_project')
from data.utils_file import pad


/home/sushen/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def ndvi(band4, band8):
    return (band8 - band4)/(band8 + band4)

def fdi(band6, band8, band10):
    l_nir = 833.0
    l_redge = 740.0
    l_swir = 1614.0
    
    r_acc = band6 + 10*(band10 - band6)*(l_nir - l_redge)/(l_swir - l_redge)
    
    return band8 - r_acc

In [3]:
list_shp = glob.glob("/data/sushen/marinedebris/project/floObj_shapefiles_trial_2/*.shp")
len(list_shp)

36

In [4]:
# First we create an empty GDF
gdf_all = gpd.GeoDataFrame(columns=['marida_cla', 'geometry'], geometry='geometry')

# Append every GeoDataFrame into a single one
for file_index in np.arange(len(list_shp)): 
    shp_file_path = list_shp[file_index]
    gdf_temp = gpd.read_file(shp_file_path)

    gdf_all = pd.concat([gdf_all, gdf_temp], ignore_index=True)

print(len(gdf_all))
gdf_all['marida_cla'].value_counts()


3378


Water                       1956
Marine Debris               1047
Waves                        237
Unresolved error             134
Sargassum                      2
Natural Organic Material       2
Name: marida_cla, dtype: int64

In [ ]:
gdf_all_clean = gdf_all[gdf_all['marida_cla'] != 'Unresolved error']
print(len(gdf_all_clean))

In [ ]:
imagesize = 32*10

gdf_water = gdf_all_clean[gdf_all_clean['marida_cla'] == 'Water']
gdf_debris = gdf_all_clean[gdf_all_clean['marida_cla'] == 'Marine Debris']

# Choose 0 for 'Water' and 1 for 'Marine Debris'
class_type = 1

# Dataframe to sample from
if class_type == 0:
    samples = gdf_water.sample(50, random_state = 0)
else:
    samples = gdf_debris.sample(50, random_state = 0)

title_list = []
# predicted_class_list = []

if class_type == 0:
    save_folder_path = '/data/sushen/marinedebris/project/evaluation/water/'
else:    
    save_folder_path = '/data/sushen/marinedebris/project/evaluation/marine_debris/'


for sample_index in np.arange(len(samples)):
    row = samples.iloc[sample_index]
    tif_file_path = row.img_path

    # Image title and geometry object number
    image_name = os.path.basename(tif_file_path)
    image_name = image_name.split('.tif')[0]

    gdf_scene = gdf_all_clean[gdf_all_clean['img_path'] == tif_file_path]
    gdf_scene.reset_index(inplace = True)
    geometry_object_index = gdf_scene.index[gdf_scene['geometry'] == row.geometry].tolist()
    geometry_object_index = str(geometry_object_index[0])

    if class_type == 0:
        title = 'Image: ' + str(sample_index + 1) + ' | scene: ' + image_name + ' | scene object: ' + geometry_object_index \
            + ' | prediction: Water'
    else:
        title = 'Image: ' + str(sample_index + 1) + ' | scene: ' + image_name + ' | scene object: ' + geometry_object_index \
        + ' | prediction: Marine Debris'

    title_list.append(title)

    # Path to save the image
    if class_type == 0:
        save_folder_path = '/data/sushen/marinedebris/project/evaluation/water/'
    else:    
        save_folder_path = '/data/sushen/marinedebris/project/evaluation/marine_debris/'

    image_save_path = 'image_' + str(sample_index + 1) + '.png'
    image_save_path = os.path.join(save_folder_path, image_save_path)

    # Rasterio Image opening to get the transform for Window
    with rio.open(tif_file_path, "r") as src:
        transform = src.transform

    # Window definition for the cropping
    minx, miny, maxx, maxy = row.geometry.centroid.buffer(imagesize // 2).bounds
    window = rasterio.windows.from_bounds(minx, miny, maxx, maxy, transform = transform)
    width = int(window.width)
    height = int(window.height)

    # Open the scene using the window
    with rio.open(tif_file_path, "r") as src_crop:
        transform_crop = src_crop.window_transform(window)
        image_crop = src_crop.read(window=window)

    # Remove unused bands 
    image_crop = np.delete(image_crop, [9, 10], axis = 0)

    # Mask 16x16
    mask_object = samples.iloc[[sample_index]].geometry
    mask_2d = rasterize(mask_object, all_touched=True,
                        transform=transform_crop, out_shape=(height, width))

    size_mask = np.zeros((32,32))
    size_mask[8:24, 8:24] = 1

    # Protect images and mask with padding if an geometric object too close to the boundary
    image_crop, mask_2d = pad(image_crop, mask_2d, imagesize // 10)

    # Take RGB bands
    bgr_img = image_crop[1:4]
    rgb_img = np.flip(bgr_img, axis = 0)
    rgb_img = np.moveaxis(rgb_img, (0, 1, 2), (2, 0, 1))

    # Apply masks
    red_masked = rgb_img[:,:,0]
    red_masked = np.where(mask_2d == 1, red_masked, 0)
    red_masked = np.where(size_mask == 1, red_masked, 0)

    green_masked = rgb_img[:,:,1]
    green_masked = np.where(mask_2d == 1, green_masked, 0)
    green_masked = np.where(size_mask == 1, green_masked, 0)

    blue_masked = rgb_img[:,:,2]
    blue_masked = np.where(mask_2d == 1, blue_masked, 0)
    blue_masked = np.where(size_mask == 1, blue_masked, 0)

    rgb_masked = np.dstack([red_masked, green_masked, blue_masked])

    # NDVI and FDI
    ndvi_img = ndvi(image_crop[3], image_crop[7])
    fdi_img = fdi(image_crop[5], image_crop[7], image_crop[9])

    # Plots
    fig, axs = plt.subplots(1, 4, figsize = (8,5))
    axs[0].imshow(rgb_img/np.max(rgb_img))
    axs[0].set_title('RGB', fontsize=10)
    axs[0].axis('off')

    axs[1].imshow(rgb_masked/np.max(rgb_img))
    axs[1].set_title('RGB masked (16x16)', fontsize=10)
    axs[1].axis('off')

    axs[2].imshow(ndvi_img)
    axs[2].set_title('NDVI', fontsize=10)
    axs[2].axis('off')

    axs[3].imshow(fdi_img)
    axs[3].set_title('FDI', fontsize=10)
    axs[3].axis('off')

    plt.suptitle(title, y=0.75)
    plt.tight_layout()
    plt.savefig(image_save_path)
    plt.show()

# Save titles into CSV
if class_type == 0:
    save_titles_path = '/data/sushen/marinedebris/project/evaluation/water/'
else:    
    save_titles_path = '/data/sushen/marinedebris/project/evaluation/marine_debris/'

save_titles_path = os.path.join(save_titles_path, 'titles.csv')

df_title = pd.DataFrame(title_list)
df_title.to_csv(save_titles_path, index=False)

In [ ]:
# Add title and predicted class list to pandas DF and export to EXCEL for easy copy-paste to the spreadsheet
# print(title_list)



In [ ]:
if class_type == 0:
    save_folder_path = '/data/sushen/marinedebris/project/evaluation/water/'
else:    
    save_folder_path = '/data/sushen/marinedebris/project/evaluation/marine_debris/'

image_save_path = 'image_' + str(sample_index + 1) + '.png'
image_save_path = os.path.join(save_folder_path, image_save_path)
print(image_save_path)

In [ ]:
# Plots
fig, axs = plt.subplots(1,4, figsize = (8,5))
axs[0].imshow(rgb_img/np.max(rgb_img))
axs[0].set_title('RGB', fontsize=10)
axs[0].axis('off')

axs[1].imshow(rgb_masked/np.max(rgb_img))
axs[1].set_title('RGB masked (16x16)', fontsize=10)
axs[1].axis('off')

axs[2].imshow(ndvi_img)
axs[2].set_title('NDVI', fontsize=10)
axs[2].axis('off')

axs[3].imshow(fdi_img)
axs[3].set_title('FDI', fontsize=10)
axs[3].axis('off')

plt.suptitle(title, y=0.75)
plt.tight_layout()
plt.savefig(image_save_path)
plt.show()